In [7]:
## Tasseled_Cap_Summary.py
'''
This file contains a set of python functions to calculate a tasselled cap index summary. 
The summary is the percentage of scenes during the epoch in which the index has exceeded 
a set threshold. The thresholds are set to default values within this function and can be
adjusted.  

This file requires the BandIndices.py python script from dea-notebooks/Scripts

Last modified: June 2018
Authors: Bex Dunn

'''
import dask
import os
import sys
import xarray as xr


# Import external functions from dea-notebooks
sys.path.append(os.path.expanduser('~/dea-notebooks/Scripts/'))
from BandIndices import tasseled_cap

In [5]:
#set up thresholds for exceedence 
wetness_threshold = -400
greenness_threshold = 
brightness_threshold


#make dictionary of thresholds for wetness, brightness and greenness
    
    
    
    analysis_threshold = {'wetness_threshold': wetness_threshold,
                          'greenness_threshold': greenness_threshold,
                          'brightness_threshold': brightness_threshold}


def calc_wetveg_overthresh(wetness,threshold=-400):
    '''Calculate the wetness values where wetness>threshold. Inputs are wetness array and threshold value, 
    default threshold is -400. Band for wetness>threshold is added to wetness. This is not the count.'''
    if wetness is not None:
        with warnings.catch_warnings():
            #suppress irritating behaviour in xarray.where
            warnings.simplefilter("ignore")
            #water_plus_wetveg is wetness values where wetness>threshold
            wetness['water_plus_wetveg'] = wetness.wetness.where(wetness.wetness>threshold)
            print('thresholded wetness added to array')
            return wetness
    else:
        print('did not calculate wetness overthreshold' )
        return None    

In [6]:
def count_wets(wetness):
    '''count the number of wetness scenes for each pixel,
    count the amount of times that water plus wet veg is above the threshold
    load both into memory (this assumes you are using dask),
    return a dictionary of wet count and threshold count'''
    if wetness is not None:
        #count the number of wetness scenes for each pixel
        wet_count = wetness.wetness.count(dim='time')

        #count the amount of times that water plus wet veg is above the threshold
        threshold_count= wetness.water_plus_wetveg.count(dim='time')
        
        #bring both counts into memory
        wet_count.load()
        threshold_count.load()
        
        #define dictionary of wet count and threshold count
        counts = {'wet count':wet_count, 'threshold count':threshold_count}
        print('counted')
        return counts
    else:
        print('did not count' )
        return None    

In [ ]:
def count_tcindex_1sensor(over_threshold, tcindex, tci_over_threshold):
    '''count the number of tci-flagged scenes for each pixel,
    count the amount of times that tci value is above the set threshold
    load both into memory (this assumes you are using dask),
    return a dictionary of total count and threshold count
    ###FIXME currently only runs over 1 sensor!!!'''
    if over_threshold is not None:
        #count the number of flagged scenes for each pixel
        total_count = over_threshold[tcindex].count(dim='time')

        #count the amount of times that water plus wet veg is above the threshold
        threshold_count= over_threshold[tci_over_threshold].count(dim='time')
       
        #define dictionary of total count and threshold count
        counts = {'total count':total_count, 'threshold count':threshold_count}
        
        count_of_ex = counts['threshold count']/counts['total count']
        print('counted')
        return count_of_ex, counts
    else:
        print('did not count')
        return None      